In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import streamlit as st

## 💶 Coûts des Aliments
- Prix des Aliments
- Cout en CO2


In [27]:
cout_co2 = pd.read_csv("co2_couts.csv")
cout_co2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Nom                       35 non-null     object 
 1   Kg CO2 pour 1 kilo ou 1L  35 non-null     float64
 2   Portion 100G              35 non-null     float64
 3   Prix Unitaire Kg          35 non-null     object 
dtypes: float64(2), object(2)
memory usage: 1.2+ KB


## 🍽️ Composition des repas

- Code_entrée, Entrée
- Code_plat, Plat
- Code_légumes, Légumes
- Code_laitage, Laitage
- Code_dessert, Dessert
- Code_gouter, Gouter / Code_gouter_02, Gouter_02

## 🌦️ Conditions météorologiques

- Température
- Humidité
- Vitesse du vent moyen 10 mn

Cela joue un rôle sur l'appetit des gens ( en été pas de repas lourd alors que qu'en hiver cela est plus préféré)

## ⏳Facteurs logistiques

- Attente moyenne → Un temps d’attente trop long peut décourager les convives et entraîner plus de gaspillage.
- Allergies → Certains plats peuvent être évités et donc plus gaspillés.

In [28]:
data_meteo = pd.read_csv("data-meteo.csv")
print(data_meteo.columns)
data_meteo.fillna('0' , inplace=True)
print(data_meteo.isna().sum())

Index(['Index', 'Date', 'Secteur', 'Commentaire semaine', 'Commentaire jour',
       'Code_entrée', 'Entrée', 'Code_plat', 'Plat', 'Code_légumes', 'Légumes',
       'Code_laitage', 'Laitage', 'Code_dessert', 'Dessert', 'Code_gouter',
       'Gouter', 'Code_gouter_02', 'Gouter_02', 'Taux participation',
       'Température', 'Humidité', 'Vitesse du vent moyen 10 mn',
       'Taux gaspillage', 'Attente moyenne', 'Allergies'],
      dtype='object')
Index                          0
Date                           0
Secteur                        0
Commentaire semaine            0
Commentaire jour               0
Code_entrée                    0
Entrée                         0
Code_plat                      0
Plat                           0
Code_légumes                   0
Légumes                        0
Code_laitage                   0
Laitage                        0
Code_dessert                   0
Dessert                        0
Code_gouter                    0
Gouter                

/var/folders/rh/hfxf5jxd4ql1r3b72l30nxt80000gn/T/ipykernel_11637/144975104.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data_meteo.fillna('0' , inplace=True)


In [ ]:
data_meteo["Date"] = pd.to_datetime(data_meteo["Date"])
data_meteo["Year"] = data_meteo["Date"].dt.year
data_meteo["Month"] = data_meteo["Date"].dt.month
data_meteo["Day"] = data_meteo["Date"].dt.day

data_meteo["Taux gaspillage"] = pd.to_numeric(data_meteo["Taux gaspillage"])
data_meteo = data_meteo.dropna()

columns_to_encode=["Entrée" , "Dessert" , "Laitage" , "Légumes" , "Plat" , "Secteur" , "Allergies", "Gouter", "Gouter_02" ]

existing_columns = [col for col in columns_to_encode if col in data_meteo.columns]
data_meteo = pd.get_dummies(data_meteo, columns=existing_columns)

y = data_meteo['Taux gaspillage']
X = data_meteo.drop(columns=["Taux gaspillage" ,"Date" ,  "Commentaire semaine" , "Commentaire jour",
                             "Code_gouter", "Code_gouter_02" , "Code_entrée" , "Code_entrée" , "Code_plat" , "Code_légumes" , "Code_laitage"
                             , "Code_dessert"])

print(X.info())


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor(random_state=42))
])

param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [None, 10, 20],
    'model__min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_

# Make predictions
y_pred = best_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'Best Parameters: {grid_search.best_params_}')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5137 entries, 0 to 5136
Columns: 3137 entries, Index to Gouter_02_VACANCES SCOLAIRES
dtypes: bool(3128), float64(3), int32(3), int64(2), object(1)
memory usage: 15.6+ MB
None


In [21]:
data_meteo

,Index,Date,Commentaire semaine,Commentaire jour,Code_entrée,Code_plat,Code_légumes,Code_laitage,Code_dessert,Code_gouter,...,Gouter_02_Pain individuel\nBarre chocolatée,Gouter_02_Pain individuel\nBarre chocolatée\nClémentines,Gouter_02_Pain individuel\nBarre chocolatée\nPomme,Gouter_02_Pain individuel\nClémentines,Gouter_02_Pain individuel\nKiwi,Gouter_02_Pain individuel\nOrange,Gouter_02_Pain individuel\nPomme,Gouter_02_Pain individuel\nPomme\nBarre chocolatée,Gouter_02_REPAS DE RÉSERVE,Gouter_02_VACANCES SCOLAIRES
0,0,2016-03-21,0,0,FLS,VPF,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
1,1,2016-03-21,0,0,FLS,VPF,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
2,2,2016-03-21,0,0,FLS,VPF,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
3,3,2016-03-22,0,0,AB,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
4,4,2016-03-22,0,0,AB,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5132,5132,2023-01-20,0,0,BIO,DUR,0,0,BIO,BIO,...,False,False,False,False,False,False,False,False,False,False
5133,5133,2023-01-20,0,0,BIO,DUR,0,0,BIO,BIO,...,False,False,False,False,False,False,False,False,False,False
5134,5134,2023-01-23,0,0,BIO,FR / BIO / DUR,0,0,BIO,BIO,...,False,False,False,False,False,False,False,False,False,False
5135,5135,2023-01-23,0,0,BIO,FR / BIO / DUR,0,0,BIO,BIO,...,False,False,False,False,False,False,False,False,False,False


In [22]:

print(f'Mean Squared Error: {mse}')
print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')

Mean Squared Error: 0.0189841030857645
Coefficients: [ 4.55450827e-05  3.49280789e-02 -6.51368056e-04 ...  3.60376451e-02
 -7.30727501e-03 -2.08851353e-03]
Intercept: 55.94434476938744


In [23]:
st.title("Prédiction du Taux de Gaspillage")


year = st.number_input("Année", min_value=2000, max_value=2100, value=2023)
month = st.number_input("Mois", min_value=1, max_value=12, value=1)
day = st.number_input("Jour", min_value=1, max_value=31, value=1)
entree = st.text_input("Entrée", "Pamplemousse")
plat = st.text_input("Plat", "Rôti de porc au jus")
legumes = st.text_input("Légumes", "Lentilles")
laitage = st.text_input("Laitage", "Crème au chocolat")
gouter = st.text_input("Goûter", "Pain/ confiture d'abricots/ lait")
allergies = st.number_input("Allergies", min_value=0.0, max_value=1.0, value=0.02)
taux_participation = st.number_input("Taux de participation", min_value=0.0, max_value=1.0, value=0.01)
temperature = st.number_input("Température", value=10.7)
humidite = st.number_input("Humidité", value=58)
vent = st.number_input("Vitesse du vent moyen 10 mn", value=4.5)
attente = st.number_input("Attente moyenne", value=0)

nouvelle_observation = {
    "Year": year,
    "Month": month,
    "Day": day,
    "Entrée": entree,
    "Plat": plat,
    "Légumes": legumes,
    "Laitage": laitage,
    "Gouter": gouter,
    "Allergies": allergies,
    "Taux participation": taux_participation,
    "Température": temperature,
    "Humidité": humidite,
    "Vitesse du vent moyen 10 mn": vent,
    "Attente moyenne": attente,
    # Ajouter ici toutes les valeurs des variables catégorielles (avec one-hot encoding)
}

nouveau_df = pd.DataFrame([nouvelle_observation])
nouveau_df = nouveau_df.reindex(columns=X_train.columns, fill_value=0)

if st.button("Prédire"):
    prediction = model.predict(nouveau_df)
    st.write(f"Prédiction du taux de gaspillage : {prediction[0]:.2f}%")


2025-03-01 11:34:42.449 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 11:34:42.450 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 11:34:42.451 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 11:34:42.452 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 11:34:42.452 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 11:34:42.453 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 11:34:42.454 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 11:34:42.454 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [24]:
# Prédiction
